In [ ]:
HTML(read(open("style.css"), String))

# Advanced Board Struct
This notebook implements the struct `AdvBoard` which combines additional attributes to the `Board` class of the chess.jl Library. These attributes relate to the `Board` directly meaning that a `Board` always has the same value for this attribute during any part of the calculation. Additionally functions containing the Advanced Board are implemented in this notebook.

Added Attributes:

1. `score::Int64` is the static evaluation of the board in centipawns
1. `hash::UInt64` is the hash of the `Board`
1. `repCounter::Dict{UInt64, Int8}` is a dictionary counting how often a position has been reached. This is needed to check whether a 3-fold-repetition has occurred resulting in a draw.

All of these attributes are not implemented by the Chess.jl Library. The `score` and `hash` are implemented using our choice of algorithms and therefore are not accounted for. The `repCounter` dictionary was added because the `Board` object only contains the information about the current position and does not have any information about the game. The important information is whether a threefold repetition has occurred which leads to an immediate draw.

In [ ]:
using Chess
using NBInclude

In [ ]:
@nbinclude("Repetition.ipynb")

In [ ]:
@nbinclude("ZobristHashing.ipynb")

In [ ]:
@nbinclude("EvaluatePosition.ipynb")

This notebook contains the `Advanced Board` (short: `AdvBoard`) struct which bundles additional information about the board to the `Board`. As the `::Board` Class of Chess.jl does not contain all the information we need to the board this will be expanded to contain the following additional contents:

1. `score::Int64` is the static evaluation of the board in centipawn
1. `hash::UInt64`  is the hash of the board
1. `repCounter::Dict{UInt64, Int8}` is a Dictionary that counts how often a position as occurred

The choice for a `mutual struct` was made after checking the source code of the Chess.jl implementation of the regular `Board`.

In [ ]:
mutable struct AdvBoard
    state::Board
    score::Int64
    hash::UInt64
    repCounter::Dict{UInt64, Int8}
end

### Constructor for AdvBoard

The Constructor `AdvBoard` creates an `AdvBoard` from a `Board` and initializes all attributes automatically.

Arguments:

1. `board::Board` is the board that the AdvBoard should be representing

Returns an `AdvBoard` with calculated attributes.

In [ ]:
function AdvBoard(board::Board)::AdvBoard
    score = evaluate_position(board)
    hash  = zobrist_hash(board)
    repCounter = Dict{UInt64, Int8}()
    incrementHash!(repCounter, hash)
    return AdvBoard(board, score, hash, repCounter)
end

### domoveAdv! function 

The function `domoveAdv!` has the same functionality for the `AdvBoard` as the `domove!` function for a `Board`. It applies the given move on the AdvBoard and changes all attributes according to the new position.

Arguments:

1. `aBoard::AdvBoard` is the chess board where the move should be applied
1. `move::Move` is the move that should be done

Returns a three-tuple including information needed to undo the move. These values should not be mutated.

In [ ]:
function domoveAdv!(aBoard::AdvBoard, move::Move)::Tuple{Int64, UInt64, UndoInfo}
    oldscore::Int64 = aBoard.score
    oldhash::UInt64 = aBoard.hash
    aBoard.score = evaluate_move(aBoard, move) 
    aBoard.hash = zobrist_hash(aBoard.state, aBoard.hash, move) 
    undoinfo = domove!(aBoard.state, move)
    incrementHash!(aBoard.repCounter, aBoard.hash)
    return (oldscore, oldhash, undoinfo)
end

In [ ]:
function domoveAdvNoScore!(aBoard::AdvBoard, move::Move)::Tuple{UInt64, UndoInfo}
    oldhash::UInt64 = aBoard.hash
    aBoard.hash = zobrist_hash(aBoard.state, aBoard.hash, move) 
    undoinfo = domove!(aBoard.state, move)
    incrementHash!(aBoard.repCounter, aBoard.hash)
    return (oldhash, undoinfo)
end

### undomoveAdv! function 

The function `undomoveAdv!` has the same functionality for the `AdvBoard` as the `undomove!` function for a `Board`. It undoes the last move done on  the AdvBoard and changes all attributes according to the new position.

Arguments:

1. `aBoard::AdvBoard` is the chess board where the move should be applied
1. `undoinfo` is the three-tuple returned from the `domoveAdv!` function

In [ ]:
function undomoveAdv!(ABoard::AdvBoard, undoinfo::Tuple{Int64, UInt64, UndoInfo})
    decrementHash!(ABoard.repCounter, ABoard.hash)
    undomove!(ABoard.state, undoinfo[3])
    ABoard.score = undoinfo[1]
    ABoard.hash  = undoinfo[2]
end

In [ ]:
function undomoveAdvNoScore!(ABoard::AdvBoard, undoinfo::Tuple{UInt64, UndoInfo})
    decrementHash!(ABoard.repCounter, ABoard.hash)
    undomove!(ABoard.state, undoinfo[2])
    ABoard.hash  = undoinfo[1]
end

In [ ]:
function clearRepCounter!(aBoard::AdvBoard)
    aBoard.repCounter = Dict{UInt64, Int8}()
end

### evaluate_move function

In order to implement repetition control in the isterminal function, it is necessary to call the function with the Advanced Board. For this reason, we have overloaded the evaluate_move function to take an Advanced Board as input. The `evaluate_move` function takes in an advanced chess board `aBoard` and a `move` and returns the score of the board after performing the move. The function uses an incremental implementation, which means it only considers and calculates the differences between the old and new position.

The implementation of the evaluate_move function begins by performing the move on the board using the domove! function and storing the UndoInfo object returned by the function. The function then checks if the board is in a terminal state by calling the isterminal function with the Advanced Board as input. If the board is in a terminal state, the function returns the score of the board as evaluated by the terminal_evaluation function. Otherwise, the function calls the evaluate_move function with the board and the move as input parameters.

In [ ]:
function evaluate_move(aBoard::AdvBoard, move::Move)::Int64
    undoinfo::Tuple{UInt64, UndoInfo} = domoveAdvNoScore!(aBoard, move)
    if isterminal(aBoard.state)
        score::Int64 = terminal_evaluation(aBoard)
        undomoveAdvNoScore!(aBoard, undoinfo)
        return score
    end
    if isRepetition(aBoard.repCounter)
        undomoveAdvNoScore!(aBoard, undoinfo)
        return 0
    end
    undomoveAdvNoScore!(aBoard, undoinfo)
    return evaluate_move(aBoard.state, move, aBoard.score)
end

### terminal_evaluation function

The terminal_evaluation function in Julia takes a Board object as an argument and returns an integer score representing the evaluation of the board in a given terminal state. The function checks if the board is in checkmate, stalemate, material draw, or rule 50 draw, and returns a score accordingly.

The function first checks if the board is in a checkmate position. If the board is in checkmate, the function returns a score of -100000 or 100000, depending on which player is in checkmate.
A repetition position occurs when the same position occurs three times in a game, and the same player has the same possible moves. If the board is a draw, it returns a score of 0.

In [ ]:
function terminal_evaluation(aBoard::AdvBoard)::Int64
    if ischeckmate(aBoard.state)
        return sidetomove(aBoard.state) == WHITE ? -100000 : 100000
    elseif isstalemate(aBoard.state) || ismaterialdraw(aBoard.state) || isrule50draw(aBoard.state)
        return 0
    end
end